In [ ]:
# from langchain_community.embeddings import OllamaEmbeddings
from langchain.llms import Ollama
from langchain.agents import AgentExecutor,create_react_agent
from langchain_core.prompts import ChatPromptTemplate

# from langchain.prompts import PromptTemplate
from langchain.tools import tool
from langchain import hub

llm = Ollama(model="mistral")


In [7]:

# prompt = hub.pull("hwchase17/openai-tools-agent")
# prompt = hub.pull("hwchase17/react")

from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("""
Answer using this EXACT format:

Question: {input}
Thought: [your reasoning]
Action: multiply|add|exponentiate
Action Input: {{"first_int": number, "second_int": number}}
Observation: [result]
Final Answer: [final response]

Begin!

Question: {input}
Thought:{agent_scratchpad}
""")

prompt.pretty_print()


Answer using this EXACT format:

Question: {input}
Thought: [your reasoning]
Action: multiply|add|exponentiate
Action Input: {"first_int": number, "second_int": number}
Observation: [result]
Final Answer: [final response]

Begin!

Question: {input}
Thought:{agent_scratchpad}



In [8]:
@tool
def add(first_int: int, second_int: int) -> int:
    """Useful for adding two integers together. Example: add(2, 3) returns 5."""
    return first_int + second_int

@tool
def exponentiate(base: int, exponent: int) -> int:
    """Useful for raising a base to an exponent power. Example: exponentiate(2, 3) returns 8."""
    return base**exponent

@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers. Parameters: first_int (number), second_int (number). Example: multiply(3, 4) returns 12."""
    return first_int * second_int

tools = [multiply, add, exponentiate]

In [5]:
agent = create_react_agent(llm, tools, prompt)

# from langchain.agents import initialize_agent, AgentType
# agent = initialize_agent(
#     tools,
#     llm,
#     # agent="zero-shot-react-description",  More compatible with local models
#     verbose=True
# )

# # Test the agent
# agent.run("What is 3 multiplied by 4 plus 5?")

In [9]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True,handle_parsing_errors=True)
agent_executor.invoke({"input": "What is 3 multiply by 4?"})
# agent_executor.invoke({"input": "What is 3 multiplied by 4 plus 5?"})

# llm("3 raised to the power of 4")



> Entering new AgentExecutor chain...
 I need to find the product of two numbers. In this case, I am given the numbers 3 and 4. The tool that will be useful here is `multiply`.

Action: multiply
Action Input: (3, 4)

ValidationError: 2 validation errors for multiply
first_int
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='(3, 4)', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/int_parsing
second_int
  Field required [type=missing, input_value={'first_int': '(3, 4)'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing